# 🎥 Colab Video Compressor

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AdittyaMondal/ColabVideoCompressor/blob/main/colab_notebook.ipynb)

Advanced video compression bot with GPU acceleration and multiple presets.

In [ ]:
#@title 🚀 Configure and Run Your GPU-Accelerated Bot
#@markdown ### 1. Telegram API Credentials
APP_ID = ""  #@param {type:"string"}
API_HASH = ""  #@param {type:"string"}
BOT_TOKEN = "" #@param {type:"string"}
OWNER = "" #@param {type:"string"}
THUMBNAIL = "https://graph.org/file/75ee20ec8d8c8bba84f02.jpg" #@param {type:"string"}

#@markdown --- 
#@markdown ### 2. Select Compression Preset
#@markdown Choose a preset. 'Balanced' is recommended for a good mix of speed and quality.
preset_choice = "Balanced (720p)" #@param ["Ultra Fast (480p)", "Fast (720p)", "Balanced (720p)", "High Quality (1080p)", "Custom"]

#@markdown --- 
#@markdown ### 3. Custom Preset (Only if 'Custom' is selected above)
#@markdown These settings will be ignored unless you choose 'Custom'.
custom_preset_speed = "medium" #@param ["fast", "medium", "slow", "hq"]
custom_quality_qp = 28 #@param {type:"slider", min:18, max:35, step:1}
custom_resolution = "720" #@param ["480", "720", "1080", "-1"]
custom_audio_bitrate = "128k" #@param ["96k", "128k", "192k", "256k"]

#@markdown --- 
#@markdown ### 4. Advanced Settings
MAX_FILE_SIZE = 3072 #@param {type:"integer"}
MAX_QUEUE_SIZE = 10 #@param {type:"integer"}

import os
from IPython.display import HTML, display
import subprocess

# Define preset parameters
PRESET_PARAMS = {
    "Ultra Fast (480p)":   {"preset": "fast",   "qp": 34, "scale": 480,  "ab": "96k"},
    "Fast (720p)":         {"preset": "fast",   "qp": 30, "scale": 720,  "ab": "128k"},
    "Balanced (720p)":     {"preset": "medium", "qp": 28, "scale": 720,  "ab": "128k"},
    "High Quality (1080p)":{"preset": "slow",   "qp": 24, "scale": 1080, "ab": "192k"}
}

if preset_choice == "Custom":
    try:
        scale_value = int(custom_resolution)
    except:
        scale_value = -1
    params = {
        "preset": custom_preset_speed,
        "qp": custom_quality_qp,
        "scale": scale_value,
        "ab": custom_audio_bitrate
    }
else:
    params = PRESET_PARAMS[preset_choice]

def run_shell(cmd):
    """Helper to run shell commands cleanly."""
    process = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if process.returncode != 0:
        print(f"❌ Error running command: {cmd}")
        print(process.stderr)
    else:
        print(process.stdout.strip())

def setup_bot():
    print("🚀 Starting bot setup...")
    if not all([APP_ID, API_HASH, BOT_TOKEN, OWNER]):
        raise ValueError("❌ All fields in 'Telegram API Credentials' are required!")

    if not os.path.exists('/content/ColabVideoCompressor'):
        print("Cloning repository...")
        run_shell("git clone -q https://github.com/AdittyaMondal/ColabVideoCompressor.git")

    os.chdir("/content/ColabVideoCompressor")

    print("Installing system packages...")
    run_shell("apt-get -qq update")
    run_shell("apt-get -qq install -y ffmpeg mediainfo")

    print("Creating directories...")
    os.makedirs("encode", exist_ok=True)
    os.makedirs("downloads", exist_ok=True)
    os.makedirs("thumb", exist_ok=True)
    run_shell("chmod -R 777 encode downloads thumb")

    print("⚙️ Configuring environment with selected parameters...")
    env_content = f"""APP_ID={APP_ID}
API_HASH={API_HASH}
BOT_TOKEN={BOT_TOKEN}
OWNER={OWNER}
THUMBNAIL='{THUMBNAIL}'
MAX_FILE_SIZE={MAX_FILE_SIZE}
MAX_QUEUE_SIZE={MAX_QUEUE_SIZE}
# Encoding parameters set by the notebook
V_PRESET={params['preset']}
V_QP={params['qp']}
V_SCALE={params['scale']}
A_BITRATE='{params['ab']}'
"""

    with open('.env', 'w') as f:
        f.write(env_content)

    print("Installing Python dependencies...")
    run_shell("pip install -q -r requirements.txt")

    print("🤖 Bot is starting... Check your bot on Telegram!")
    run_shell("python3 -m bot")

#@markdown --- 
#@markdown ### 5. Run Setup and Start Bot
run_setup = True #@param {type:"boolean"}

if run_setup:
    try:
        setup_bot()
    except Exception as e:
        print(f"❌ An error occurred during setup: {str(e)}")

# 📋 Display Notes
display(HTML('''
<div style='background-color: #f0f8ff; padding: 15px; border-radius: 8px; border: 1px solid #d1e7fd; margin-top: 20px;'>
<h3>📋 Important Notes:</h3>
<ul>
    <li><strong>Credentials:</strong> Get your <code>APP_ID</code> and <code>API_HASH</code> from <a href='https://my.telegram.org' target='_blank'>my.telegram.org</a>.</li>
    <li><strong>BOT_TOKEN:</strong> Get from @BotFather on Telegram.</li>
    <li><strong>OWNER ID:</strong> Get from @userinfobot (just send /start to it).</li>
    <li><strong>GPU is guaranteed:</strong> This new version ensures GPU is used for both decoding and encoding.</li>
    <li><strong>'-1' Resolution:</strong> In custom settings, a resolution of -1 means 'keep original resolution'.</li>
</ul>
</div>
'''))
